In [1]:
filename = "ISC-r1.CNV.bed"
data = {}
dataset = None
for line in open(filename).readlines():
    if not line.startswith('chr'):
        dataset = line.strip().split()[1].lstrip("name=")
        data[dataset] = {}
        continue
    line = line.strip().split()
    if not line[0] in data[dataset]:
        data[dataset][line[0]] = []
    data[dataset][line[0]].append((int(line[1]),int(line[2])))
    
for dataset in data.keys():
    for chrom in data[dataset]:
        data[dataset][chrom].sort()
#print (data)

In [2]:
# for exome overlap with CNVs, use gene file instead
import re
exomefilename = "CCDS_exons.20160908.txt"
exomedata = {}
exomedataset = None
for line in open(exomefilename).readlines():
    line = line.strip().split()
    if not line[0] in exomedata[exomedataset]:
        exomedata[exomedataset][line[0]] = []
    exomedata[exomedataset][line[0]].append((int(line[5])), (int(line[6])))
    
for exomedataset in exomedata.keys():
    for chrom in exomedata[exomedataset]:
        exomedata[exomedataset][chrom].sort()
    
#    if not line.startswith('#'):
#        exomedataset = line.strip().split()[1].lstrip("name=")
#        exomedata[exomedataset] = {}
#        continue


KeyError: None

In [21]:
from pprint import pprint
pprint(data)
pprint([(x, y.keys()) for x,y in data.items()])

{'delCases': {'chr1': [(2758721, 3017875),
                       (12668368, 12787139),
                       (12776194, 12983067),
                       (12780129, 12932535),
                       (12787014, 12932535),
                       (12787014, 12947384),
                       (12787138, 12924223),
                       (12787138, 12948321),
                       (12787138, 12948321),
                       (12787138, 12958453),
                       (12787138, 12958453),
                       (12787138, 13011237),
                       (30022182, 30558174),
                       (49140323, 49521424),
                       (49565963, 49969627),
                       (60561257, 60665637),
                       (72287807, 72439334),
                       (73364609, 73489369),
                       (73364893, 73488397),
                       (76013219, 76121485),
                       (79022947, 79689388),
                       (82190212, 82335285),
            

In [2]:
# for CNVs start and/or end position less than diff(100,000) bp
datamerge = {}
diff = 100000
for dataset in data.keys():
    datamerge[dataset] = {}
    for chrom in data[dataset]:
        datamerge[dataset][chrom] = [ data[dataset][chrom][0] ]
#print (data[dataset][chrom][-1][1])
        for item1, item2 in zip(data[dataset][chrom], data[dataset][chrom][1:]):
#            print (datamerge[dataset][chrom])
            diff1 = abs(item2[0] - item1[0])
            diff2 = abs(item2[1] - item1[1])
            diff3 = abs(item2[0] - datamerge[dataset][chrom][-1][0])
            diff4 = abs(item2[1] - datamerge[dataset][chrom][-1][1])
            min_start = min(item1[0], item2[0], datamerge[dataset][chrom][-1][0])
            max_end = max(item1[1], item2[1], datamerge[dataset][chrom][-1][1])
            if ( diff1 <= diff or diff2 <= diff ) and ( diff3 <= diff or diff4 <= diff ):
                datamerge[dataset][chrom][-1] = (( min_start, max_end ))
            else:
                datamerge[dataset][chrom].append((item2[0], item2[1]))
            

In [4]:
# for CNVs not overlap
datamerge2 = {}
for dataset in data.keys():
    datamerge2[dataset] = {}
    for chrom in data[dataset]:
        datamerge2[dataset][chrom] = [ data[dataset][chrom][0] ]
        for item1, item2 in zip(data[dataset][chrom], data[dataset][chrom][1:]):
            item3_0 = datamerge2[dataset][chrom][-1][0]
            item3_1 = datamerge2[dataset][chrom][-1][1]
            min_start = min(item1[0], item2[0], item3_0)
            max_end = max(item1[1], item2[1], item3_1)
            if (item1[1] >= item2[0] and item1[1] <= item2[1]) or (item1[0] <= item2[0] and item1[1] >= item2[1]) and (item1[0] <= item3_1):
                datamerge2[dataset][chrom][-1] = (( min_start, max_end ))
            else:
                datamerge2[dataset][chrom].append(( item2[0], item2[1] ))
            

In [24]:
from pprint import pprint
# pprint(datamerge)
print (datamerge["delCases"])
# pprint([(x, y.keys()) for x,y in datamerge.items()])

{'chr14': [(19892935, 20493138), (21744713, 22040168), (26623684, 26866705), (26986423, 27094285), (37569543, 41009478), (37939825, 38673787), (40915706, 41074958), (41762699, 42194930), (42042721, 43728537), (43132612, 43234621), (45687634, 45865527), (47812099, 47983250), (61784411, 62203805), (66287336, 66470394), (79536021, 79636319), (81693201, 81939864), (82186231, 82458569), (82436148, 82613522), (83901679, 84140365)], 'chr20': [(2923491, 3618946), (5209011, 5346246), (8047289, 8521295), (10820886, 10945944), (14606364, 15049481), (14795396, 15014616), (23360902, 23517251), (25333391, 25585624), (40546693, 40651487), (41032409, 41178004), (55576531, 55733059)], 'chr9': [(140575, 1175526), (263084, 389053), (511040, 788774), (4339212, 4645360), (4494448, 5500645), (6021543, 6332856), (7769772, 8084834), (8005596, 8222360), (9485226, 9644835), (9895178, 10067963), (10733436, 11464831), (10963704, 11112102), (11635592, 11924378), (11762547, 12155807), (12072380, 12292929), (1491060

In [3]:
# chromosome, start and end not share the same length, did not go through
import pandas as pd

delCases = datamerge["delCases"]
# delCases_chr = delCases.keys()
# delCases_chr = []
delCases_start = {}
delCases_end = {}

for chrom in delCases.keys():
    delCases_start[chrom] = []
    delCases_end[chrom] = []
    for position in delCases[chrom]:
        delCases_start[chrom].append(position[0])
        delCases_end[chrom].append(position[1])

print (delCases)

#dat = {'chr': delCases_start.keys(), 'start': delCases_start, 'end': delCases_end}
#dat = pd.DataFrame.from_dict(dat)

#delCtrls = datamerge["delControls"]
#dupCases = datamerge["dupCases"]
#dupCtrls = datamerge["dupControls"]


{'chr15': [(20658550, 20852215), (20940554, 21230588), (21192955, 21454381), (22214855, 22366634), (27015263, 28173704), (28173703, 30647151), (28517602, 28768338), (28675248, 30571467), (29806023, 30337473), (30245953, 30664277), (36138169, 36326780), (41672410, 41775846), (42646965, 42749025), (43781873, 43936569), (43934949, 44067481), (51463569, 51572284), (52986769, 53107910), (54565709, 54677771), (66937990, 67091701), (77646730, 77790118), (78378807, 78844743), (79781267, 80067670), (82733531, 83527719), (85706849, 85821980), (87284586, 87467739), (97504037, 97658724), (98203636, 98317160), (99897496, 100303907)], 'chr7': [(52911, 978136), (3398598, 3530549), (3759288, 4087230), (6687703, 6795451), (6795357, 6906104), (8611957, 8725692), (8732200, 9003123), (9278506, 9396362), (12162495, 12305528), (14892125, 15036832), (16474038, 16607838), (17674099, 18634682), (18374605, 18498502), (18595435, 18707719), (21150024, 21270602), (21369340, 21567956), (24246876, 24420887), (294469

In [4]:
import pandas as pd
data_df = {}
for dataset in data.keys():
    data_df[dataset] = {'chr':[], 'start':[], 'end':[]}
    for chrom in data[dataset]:
        start, end = tuple(zip(*data[dataset][chrom]))
        data_df[dataset]['chr'].extend([chrom] * len(start))
        data_df[dataset]['start'] += list(start)
        data_df[dataset]['end'] += list(end)
    data_df[dataset] = pd.DataFrame.from_dict(data_df[dataset])
print(data_df)

# example for pandasql http://blog.yhat.com/posts/pandasql-sql-for-pandas-dataframes.html
# to install: pip install pandasql
# "select name2, min(txStart), max(txEnd) from <pandas table name> group by name2"

{'dupControls':         chr        end      start
0     chr15   18466954   18276341
1     chr15   20852215   20686315
2     chr15   22385580   22276386
3     chr15   22731138   22452625
4     chr15   24306175   23401208
5     chr15   26196280   25683151
6     chr15   26500068   26161489
7     chr15   27807930   27654134
8     chr15   28386993   28278679
9     chr15   28875255   28310387
10    chr15   28837372   28402011
11    chr15   30298848   28654463
12    chr15   28875770   28675248
13    chr15   28875770   28700879
14    chr15   28875770   28700879
15    chr15   28875770   28700879
16    chr15   28875770   28700879
17    chr15   28875770   28707916
18    chr15   29013705   28707916
19    chr15   28875255   28707999
20    chr15   30231489   29727846
21    chr15   30231489   29786937
22    chr15   30231489   29796277
23    chr15   30231489   29796277
24    chr15   30231489   29796277
25    chr15   30298848   29796277
26    chr15   30337473   29796277
27    chr15   30491746   2979627

In [6]:
# http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html

import pandas as pd
from pandasql import sqldf
data_refgene = pd.read_table("refGene.txt.gz", compression='gzip', sep='\t', header = None, usecols=(1,2,4,5,12), 
                            names = ['transcript', 'chr', 'tx_start', 'tx_end', 'gene'])
query = '''
select gene, chr, min(tx_start), max(tx_end) 
from data_refgene 
group by chr, gene
'''
gene_table = sqldf(query)
gene_table = gene_table.rename(columns={"min(tx_start)": "gene_start", "max(tx_end)": "gene_end", "chr": "chrom"})
#print (gene_table)

# use the single CNV dataset delCases for testing
testCNVdata = pd.DataFrame.from_dict(data_df["delCases"])
print (testCNVdata)
debug = 0
query = '''
SELECT cnv.chr, cnv.start, cnv.end, tx.gene {}
FROM testCNVdata cnv LEFT JOIN gene_table tx 
WHERE cnv.chr == tx.chrom 
AND (
(cnv.start >= tx.gene_start AND cnv.start <= tx.gene_end)
OR
(cnv.end >= tx.gene_start AND cnv.end <= tx.gene_end)
)
'''.format(", tx.chrom, tx.gene_start, tx.gene_end" if debug else '')
testcmb = sqldf(query)
#print (testcmb)

        chr       end     start
0     chr15  20852215  20658550
1     chr15  20852215  20686315
2     chr15  21230588  20940554
3     chr15  21230588  21010643
4     chr15  21454381  21192955
5     chr15  21340985  21205735
6     chr15  22366634  22214855
7     chr15  28173704  27015263
8     chr15  28360646  28173703
9     chr15  28360646  28173703
10    chr15  28377090  28173703
11    chr15  30647151  28173703
12    chr15  28370729  28231568
13    chr15  28370729  28231568
14    chr15  28768338  28517602
15    chr15  28767234  28519571
16    chr15  28767234  28563286
17    chr15  30231489  28675248
18    chr15  30231489  28675248
19    chr15  30571467  28692551
20    chr15  28875255  28702515
21    chr15  30248707  28707916
22    chr15  30248707  28707916
23    chr15  30491746  28723577
24    chr15  30571467  28723577
25    chr15  30337473  29806023
26    chr15  30664277  30245953
27    chr15  30405736  30276166
28    chr15  30402926  30291556
29    chr15  30655447  30297184
...     

In [8]:
query = '''
select gene, chr, min(tx_start), max(tx_end) 
from data_refgene 
where gene = 'FRG1DP'
group by chr, gene
'''
sqldf(query)

,gene,chr,min(tx_start),max(tx_end)
0,FRG1DP,chr20,29612047,29628331
1,FRG1DP,chrUn_gl000219,83212,99477


In [9]:
query = '''
SELECT gene, count(*)
FROM testcmb
GROUP BY gene, chr
'''
# count how many cnv in each gene
cnv_count_table = sqldf(query)
print (cnv_count_table)

          gene  count(*)
0      AADACL3        11
1         AAK1         1
2        ABCC1         1
3        ABCC4         1
4        ABCC6         1
5       ABHD12         2
6          ABR         1
7       ACTR3B         1
8      ADAMTS3         1
9    ADAMTS7P1         2
10       ADAP1         1
11       ADCY3         1
12      ADGRL2         2
13        AFF1         1
14       AGAP1         1
15       AGBL1         1
16       AGBL4         2
17      AKR1C3         2
18        AKT3         1
19       ALPK2         1
20        AMBN         3
21   ANKRD20A2         1
22   ANKRD20A3        40
23     ANKRD39         1
24       APOBR         1
25        AREG         1
26        ASPA         1
27       ASTN1         1
28       ASTN2         1
29      ATP11A         2
..         ...       ...
583      USH2A         1
584   USP17L12         2
585   USP17L20         2
586   USP17L21         2
587   USP17L22         2
588     USP6NL         1
589       VAV3         2
590     VPS13B         2


In [36]:
# use the four CNV datasets
import pandas as pd
from pandasql import sqldf
CNVdata = data_df
cnv_in_genes = {}
cnv_counts = {}
for name, value in CNVdata.items():
    query = '''
    SELECT cnv.chr, cnv.start, cnv.end, tx.gene
    FROM value cnv LEFT JOIN gene_table tx
    WHERE cnv.chr == tx.chrom 
    AND (
    (cnv.start >= tx.gene_start AND cnv.start <= tx.gene_end)
    OR
    (cnv.end >= tx.gene_start AND cnv.end <= tx.gene_end)
    )
    '''
    cnv_in_genes[name] = value = sqldf(query)
    query = '''
    SELECT gene, count(gene), chr, start, end
    FROM value
    GROUP BY gene
    ORDER BY count(gene) DESC
    '''
    cnv_counts[name] = sqldf(query).drop_duplicates(subset = ('chr', 'start', 'end'))
#print (cnv_in_genes)
print (cnv_counts)

{'delControls':              gene  count(gene)    chr      start        end
0       MIR7641-2           69  chr12   97917462   98172528
1         MIR4454           59   chr5  127773854  127899486
2         MIR5096           39   chr1  224703949  224804206
3       ANKRD20A3           26   chr9   67136713   67715387
9         FAM74A1           24   chr9   64618429   65382019
10     NUTM2B-AS1           24  chr10   81475459   81607601
14      LOC286297           22   chr9   66971754   67762910
15        MACROD2           21  chr20   14887620   15085155
16         IMMP2L           20   chr7  110966922  111429782
17      MIR4435-1           20   chr2  110197433  110854129
21   LOC102724238           18   chr9   44330473   44577961
23        SRGAP2D           18   chr1  193328314  193481519
24          DDX60           17   chr4  169197659  169367172
26         LINGO2           17   chr9   28724248   28837171
27      MIR4509-1           17  chr15   28733953   28875255
30        UGT2B10       

In [35]:
http://seaborn.pydata.org/examples/horizontal_barplot.html
https://pypi.python.org/pypi/fisher/

SyntaxError: invalid syntax (<ipython-input-35-f7972460e58a>, line 1)

In [43]:
df = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
print (df)
df2=df.rename(index=str, columns={"A": "a", "B": "c"})
print (df2)

   A  B
0  1  4
1  2  5
2  3  6
   a  c
0  1  4
1  2  5
2  3  6


In [28]:
%load_ext rpy2.ipython

In [35]:
%%R
tmp = matrix(c(20,7,200050,199980),2,2, byrow=T)
fisher.test (tmp)


	Fisher's Exact Test for Count Data

data:  tmp
p-value = 0.01915
alternative hypothesis: true odds ratio is not equal to 1
95 percent confidence interval:
 1.160085 7.994066
sample estimates:
odds ratio 
  2.856301 



In [ ]:
import numpy as np
a = np.arange(15).reshape(3, 5)

In [16]:
#pip install fisher
#pip install --upgrade pip
#pip install git+https://github.com/brentp/fishers_exact_test.git
from fisher import pvalue
